### imports

In [3]:
import requests
import json
import ee
import geemap
import folium
import os
import pandas as pd

### Web app deployed URLs

In [4]:
asset_registry_base = "https://api-ar.agstack.org"
user_registry_base = "https://user-registry.agstack.org"

### Add your email & password to register with Agstack and test out the APIs

In [7]:
email = "remi.dannunzio@fao.org" #"my_name@domain.org"
phone_num = "0123456789"
password = "Remi12345" #"this_is_not_a_password"
discoverable = True

#### Start Session

In [6]:
# using session to store cookies that are persistent
session = requests.session()
session.headers = headers = {
    'Accept': 'application/json',
    'Content-Type': 'application/json'
}
req_body = {'email': email, 'password': password}
res = session.post(user_registry_base, json=req_body)
print(session.cookies)
print(res.status_code)

<RequestsCookieJar[]>
500


### 1. Grab land cover datasets

In [8]:
ee.Initialize()

#### Fetch: Tropical Moist Forest by JRC
Link:
Overview:
Reference:

Legend: 
- value 10. Undisturbed Tropical Moist Forest (TMF) 
- value 20. Degraded TMF 
- value 30. TMF regrowth 
- value 41. Deforested land - Forest converted to tree plantations 
- value 42. Deforested Land - Forest converted to water
- value 43. Deforested Land - Forest converted to other LC
- value 50. Ongoing deforestation or degradation (2019-2021)
- value 60. Permanent or Seasonal Water  (Pekel et al. 2016 & updates for years 2015-2021) 
- value 70. Other land cover (including afforestation)

In [9]:
TransitionMap = ee.ImageCollection('projects/JRC/TMF/v1_2021/TransitionMap_Subtypes').mosaic()

TransitionMap_Main = TransitionMap.where((TransitionMap.gte(10))and(TransitionMap.lte(12)), 10)
TransitionMap_Main = TransitionMap_Main.where((TransitionMap.gte(21))and(TransitionMap.lte(26)), 20)
TransitionMap_Main = TransitionMap_Main.where((TransitionMap.gte(61))and(TransitionMap.lte(62)), 20)
TransitionMap_Main = TransitionMap_Main.where((TransitionMap.gte(31))and(TransitionMap.lte(33)), 30)
TransitionMap_Main = TransitionMap_Main.where((TransitionMap.gte(63))and(TransitionMap.lte(64)), 30)
TransitionMap_Main = TransitionMap_Main.where((TransitionMap.gte(81))and(TransitionMap.lte(86)), 41)
TransitionMap_Main = TransitionMap_Main.where((TransitionMap.gte(73))and(TransitionMap.lte(74)), 42)
TransitionMap_Main = TransitionMap_Main.where((TransitionMap.gte(41))and(TransitionMap.lte(42)), 43)
TransitionMap_Main = TransitionMap_Main.where((TransitionMap.gte(65))and(TransitionMap.lte(66)), 43)
TransitionMap_Main = TransitionMap_Main.where((TransitionMap.gte(51))and(TransitionMap.lte(54)), 50)
TransitionMap_Main = TransitionMap_Main.where((TransitionMap.eq(67)),50)
TransitionMap_Main = TransitionMap_Main.where((TransitionMap.gte(71))and(TransitionMap.lte(72)), 60)
TransitionMap_Main = TransitionMap_Main.where((TransitionMap.gte(91)), 70)

In [10]:
tmf = TransitionMap_Main

#### Fetch FDAP Palm probability layer
Link: 
Overview:

In [11]:
fdap_palm = ee.Image('projects/fao-fdp-review/assets/palm/palm_prob_test')

#### Fetch GFC tree cover, derive tree cover in 2020
Link: 
Overview:

#### Fetch: ESRI 2020 Global Land Use Land Cover from Sentinel-2
- Link: https://gee-community-catalog.org/projects/esrilc2020/
- Overview: 
- Reference:  Karra, Kontgis, et al. “Global land use/land cover with Sentinel-2 and deep learning.” IGARSS 2021-2021 IEEE International Geoscience and Remote Sensing Symposium. IEEE, 2021.
- Legend:
1. 	No Data #FFFFFF
2. 	Water #1A5BAB
3. 	Trees #358221
4. 	Grass #A7D282
5. 	Flooded Vegetation #87D19E
6. 	Crops #FFDB5C
7. 	Scrub/Shrub #EECFA8
8. 	Built Area #ED022A
9. 	Bare Ground #EDE9E4
10. 	Snow/Ice #F2FAFF
11. 	Clouds #C8C8C8

In [12]:
esri_2020 = ee.ImageCollection("projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m").mosaic()

### 2. Fetch a Field (public)

#### Cote d'Ivoire: 
0520cfac98fbc1bd7952b1c07a9f6983b83625722b6f665ea83ac9aad3512918

b84f55de2b7f3c77d1cbeb8b026a1b29be42d8b08d92058c9143e0556456820f

b7c15efb6e3c63fcfe649a2d994973a6f5caa844f720f0edb7cf24f6a6c3c1b3

fa2aff0d60cf1bc0e1f1dd4b91daf932940c31c021ca1b84f5b9445855eef02f

#### Ghana:
88bec54ad04804f5b1fafbc131266640a129be2840fa6797cda358d7e831b907

ef2f7c46fbe4fc892fdb81f9a31c9c507b9f1e4548504247dcbbab28cf8e436c

97408ef7bdac487e4a42e4abf20492b786310889fd4b0478603e2d0004c40bfb

#### Indonesia:
c288d6c94efa9011c0e3452af9f7fa0941661377030e10d29c68764617f9816d

1a41a309ae2387f36a604c9a6c81887e64357a7f61d228758e23ef766286fcd7

1a4472dc40700ef33f931863f58d444f243d64418616678fcf85c57e1f4bbf45

8e2accea7ddbb84b7f6001e00bcb60f57f563c80633b53859993522a6f05727a


In [17]:
geo_id = "0520cfac98fbc1bd7952b1c07a9f6983b83625722b6f665ea83ac9aad3512918" #"c288d6c94efa9011c0e3452af9f7fa0941661377030e10d29c68764617f9816d" # geo id for the field to fetch

In [18]:
res = session.get(asset_registry_base + f"/fetch-field/{geo_id}?s2_index=") # s2 index are indexes for which we need S2 cell token
poly_json = res.json()['Geo JSON']['geometry']['coordinates']
print(poly_json)

[[[-4.102889299392701, 5.713707261060872], [-4.102438688278199, 5.710173648788193], [-4.101483821868897, 5.710205675583411], [-4.101086854934693, 5.710664726118069], [-4.100604057312013, 5.710344458341974], [-4.100368022918702, 5.711657555086256], [-4.100528955459596, 5.712810515599572], [-4.101483821868897, 5.713525776973704], [-4.102889299392701, 5.713707261060872]]]


In [24]:
# S2 indexes of 8 and 15 are used here for the geo data
# res = session.get(asset_registry_base + f"/fetch-field/{geo_id}?s2_index=8,15") # s2 index are indexes for which we need S2 cell tokens
# poly_wkt = res.json()['Geo Data']['wkt']
# print(poly_wkt)

#### Transform geometry into a GEE Asset

In [19]:
roi = ee.FeatureCollection(ee.Feature(ee.Geometry.Polygon(poly_json),ee.Dictionary(["geoid",geo_id])))

#### Compute statistics

In [20]:
stats = os.path.join('/home/sepal-user/fdap/poly_stats.csv') 
geemap.zonal_statistics_by_group(tmf.clip(roi), roi, stats, statistics_type='SUM', denominator=1, decimal_places=2)

Computing ... 
Generating URL ...
Please wait ...
Data downloaded to /home/sepal-user/fdap/poly_stats.csv


In [21]:
table = pd.read_csv('/home/sepal-user/fdap/tmf_stats.csv')
table

,Class_sum,system:index,geoid
0,0.0,0,c288d6c94efa9011c0e3452af9f7fa0941661377030e10...


#### Display layers

In [22]:
m = geemap.Map()

In [24]:
m.addLayer(roi,{},'field')
m.addLayer(fdap_palm,{},'Palm Probability')
m.addLayer(tmf,{},'TMF')
#m.addLayer()

m.centerObject(roi)

In [25]:
m

Map(center=[5.711940454924533, -4.101628661155702], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
draw = 

In [21]:
m.addLayer(TransitionMap_Main,{},'tmf_all')

In [ ]:
# Define the class names
class_names = ['TMF_undis', 'TMF_degrad', 'TMF_regrow', 'Defor_plantation', 'Defor_water', 'Defor_land','Defor_active','water','other_land']

# Create a function to count pixels for each class
def count_pixels(image, roi):
    counts = image.reduceRegion(
        reducer=ee.Reducer.frequencyHistogram(),
        geometry=roi,
        scale=90,
        maxPixels=1e9
    )
    
    # Convert pixel counts to a feature with properties
    feature = ee.Feature(None, counts)
    
    return feature

# Apply the count_pixels function to the classified image
pixel_counts_feature = count_pixels(tmf, roi)

# Create a feature collection with a single feature
fc_pix = ee.FeatureCollection([pixel_counts_feature])

In [ ]:
Map = geemap.Map()
Map

In [ ]:
dataset = ee.Image('USGS/NLCD/NLCD2016')
landcover = ee.Image(dataset.select('landcover'))
m.addLayer(landcover, {}, 'NLCD 2016')


In [ ]:
Map.add_legend(builtin_legend='NLCD')

In [ ]:
nlcd_stats = os.path.join('/home/sepal-user/fdap/nlcd_stats_sum.csv')  
    
# statistics_type can be either 'SUM' or 'PERCENTAGE'
# denominator can be used to convert square meters to other areal units, such as square kilimeters
geemap.zonal_statistics_by_group(landcover, states, nlcd_stats, statistics_type='SUM', denominator=1000000, decimal_places=2)

In [ ]:
print(fc_pix.getInfo())

In [ ]:
m.addLayer(fc_pix,{},'counts')

In [ ]:
# Export the results as a CSV table
task = ee.batch.Export.table.toDrive(
    collection=ee.FeatureCollection([ee.Feature(None, pixel_counts)]),
    description='PixelCounts',
    fileFormat='CSV'
)

task.start()

In [ ]:
# Create a chart
chart = geemap.chart.Feature.setChartSeriesNames(class_names).FeatureCollection(pixel_counts).barChart(
    xProperty='label',
    yProperties=class_names,
    title='Pixel Counts by Class',
    xlabel='Class',
    ylabel='Pixel Count'
)

# Display the chart
chart.renderWidget(width='800px', height='400px')

In [ ]:
# Create a chart
chart = folium.Figure(width=800, height=400)
bar_chart = folium.Bar(
    data=class_names,
    height=400,
    width=800
)
bar_chart.add_to(chart)

# Add data to the chart
for i, class_name in enumerate(class_names):
    bar_chart.add_child(folium.Marker([0, 0], icon=folium.DivIcon(html=f'<p>{class_name}: {pixel_counts[i]}</p>'))



In [ ]:
polygon = roi.geometry().getInfo()
print(polygon)

In [ ]:
req_body = {"wkt": polygon, "s2_index": "8,13",
            "threshold": 90}
res = session.post(asset_registry_base +
                   "/register-field-boundary", json=req_body)
print(res.json())

In [ ]:
#Cote d'Ivoire: 
'0520cfac98fbc1bd7952b1c07a9f6983b83625722b6f665ea83ac9aad3512918'
'b84f55de2b7f3c77d1cbeb8b026a1b29be42d8b08d92058c9143e0556456820f'
'b7c15efb6e3c63fcfe649a2d994973a6f5caa844f720f0edb7cf24f6a6c3c1b3'
'fa2aff0d60cf1bc0e1f1dd4b91daf932940c31c021ca1b84f5b9445855eef02f'

#Ghana:
'88bec54ad04804f5b1fafbc131266640a129be2840fa6797cda358d7e831b907'
'ef2f7c46fbe4fc892fdb81f9a31c9c507b9f1e4548504247dcbbab28cf8e436c'
'97408ef7bdac487e4a42e4abf20492b786310889fd4b0478603e2d0004c40bfb'

#Indonesia:
'c288d6c94efa9011c0e3452af9f7fa0941661377030e10d29c68764617f9816d'
'1a41a309ae2387f36a604c9a6c81887e64357a7f61d228758e23ef766286fcd7'
'1a4472dc40700ef33f931863f58d444f243d64418616678fcf85c57e1f4bbf45'
'8e2accea7ddbb84b7f6001e00bcb60f57f563c80633b53859993522a6f05727a'


In [ ]:
polygon = "POLYGON((74.37522137108041 31.47851268355822,74.37249624672128 31.47408400867522,74.36957800331308 31.47853098343284,74.37522137108041 31.47851268355822))"
# domain filter is optional
req_body = {
    "wkt": polygon,
    "resolution_level": 13, # resolution level for overlapping criteria
    "threshold": 90, # fields with overlap greater than the threshold are fetched
    "domain": "" # filters fields associated with a specific domain using the Authority Token
}
res = session.post(asset_registry_base + "/fetch-overlapping-fields", json=req_body)
print(res.json())

#### Fetch overlapping fields filtered by a specific domain

In [ ]:
polygon = "POLYGON((74.37522137108041 31.47851268355822,74.37249624672128 31.47408400867522,74.36957800331308 31.47853098343284,74.37522137108041 31.47851268355822))"
# domain filter is optional
req_body = {
    "wkt": polygon,
    "resolution_level": 13,
    "threshold": 10,
    "s2_index": "8,13",
    "domain": "gmail.com" # filters fields associated with gmail.com domain
}
res = session.post(asset_registry_base + "/fetch-overlapping-fields", json=req_body)
print(res.json())

#### Fetch overlapping fields filtered by a specific domain and boundary type

In [ ]:
polygon = "POLYGON((74.31419491767885 31.588479372712193,74.3137550354004 31.587017094223253,74.31616902351381 31.586651521015398,74.31653380393983 31.58815036205298,74.31419491767885 31.588479372712193))"
# domain filter is optional
req_body = {
    "wkt": polygon,
    "resolution_level": 13,
    "threshold": 10,
    "s2_index": "8,13",
    "boundary_type": "automated" # boundary type filter
}
res = session.post(asset_registry_base + "/fetch-overlapping-fields", json=req_body)
print(res.json())
print("")
print("2 fields returned with boundary type as automated. No domain filter applied")

In [ ]:
polygon = "POLYGON((74.31419491767885 31.588479372712193,74.3137550354004 31.587017094223253,74.31616902351381 31.586651521015398,74.31653380393983 31.58815036205298,74.31419491767885 31.588479372712193))"
# domain filter is optional
req_body = {
    "wkt": polygon,
    "resolution_level": 13,
    "threshold": 10,
    "s2_index": "8,13",
    "domain": "test.com", # filters fields associated with test.com domain
    "boundary_type": "automated" # boundary type filter
}
res = session.post(asset_registry_base + "/fetch-overlapping-fields", json=req_body)
print(res.json())
print("")
print("1 field returned with boundary type as automated for same previous polygon as domain filter applied")

### 4. Get Percentage overlap of two fields (public)

In [ ]:
req_body = {
    "geo_id_field_1": "6f14410edb9ae3417a85c8686a5bbbb70fa8b68ef0d69c2960a803f0db4ebf9e",
    "geo_id_field_2": "89d864f50bb864fb79fb5672f51da53991bc92e12c61f9222d5e35cd5ad52c0c"
}
res = session.post(asset_registry_base + "/get-percentage-overlap-two-fields", json=req_body)
print(res.json())

### 5. Fetch fields for a Point (protected)

In [ ]:
req_body = {
    "latitude": 31.47704430446457,
    "longitude": 74.37510786779589,
    "domain": "" # filters fields associated with test.com domain
}
res = session.post(asset_registry_base + "/fetch-fields-for-a-point", json=req_body)
print(res.json())

#### Fetch specific s2 index cell tokens

In [ ]:
req_body = {
    "latitude": 31.47704430446457,
    "longitude": 74.37510786779589,
    "s2_index": "8,13" # s2 index are indexes for which we need S2 cell tokens
}
res = session.post(asset_registry_base + "/fetch-fields-for-a-point", json=req_body)
print(res.json())

### 6. Fetch the Bounding Box Fields (protected)

In [ ]:
req_body = {
    "latitudes": "31.477779423633137 31.478602920901388 31.474320655972566 31.47543699178076",
    "longitudes": "74.3729835582561 74.38032208212084 74.37404571302599 74.38112674482531"
}
res = session.post(asset_registry_base + "/fetch-bounding-box-fields", json=req_body)
print(res.json())

#### Fetch specific s2 index cell tokens

In [ ]:
req_body = {
    "latitudes": "31.477779423633137 31.478602920901388 31.474320655972566 31.47543699178076",
    "longitudes": "74.3729835582561 74.38032208212084 74.37404571302599 74.38112674482531",
    "s2_index": "8,15" # s2 index are indexes for which we need S2 cell tokens
}
res = session.post(asset_registry_base + "/fetch-bounding-box-fields", json=req_body)
print(res.json())

### Logout (protected)

In [ ]:
res = session.get(asset_registry_base + "/logout")
print(res.json())
res = session.get(user_registry_base + "/logout", cookies=session.cookies)
session.headers.clear()

### Checking if Logged out correctly

In [ ]:
# Confirming the logout from Asset Registry by requesting a Protected route
req_body = {
    "latitude": 31.47704430446457,
    "longitude": 74.37510786779589
}
res = session.post(asset_registry_base + "/fetch-fields-for-a-point", json=req_body)
print(res.json())

### Get all Domains (public)

In [ ]:
# Fetching all the domains from the User Registry
res = session.get(asset_registry_base + "/domains")
print(res.json())